<a href="https://colab.research.google.com/github/anupriya832/stone-paper-scissors-game/blob/main/ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings('ignore')

class SolarPlantAnalyzer:
    def __init__(self, csv_file_path):
        """
        Initialize the Solar Plant Analyzer with the dataset
        """
        self.data = None
        self.site_summary = None
        self.model = None
        self.load_data(csv_file_path)

        # Updated solar panel specifications
        self.panel_efficiency = 0.23  # 23% efficiency
        self.panel_rating = 320  # 320 watts per panel
        self.panel_area = 2.0  # 2 m² per panel (typical for 320W panel)
        self.mj_to_kwh = 0.2778  # Conversion factor

    def load_data(self, csv_file_path):
        """
        Load and clean the NASA POWER dataset
        """
        print("Loading NASA POWER dataset...")

        # Read the file to find where actual data starts
        with open(csv_file_path, 'r') as file:
            lines = file.readlines()

        # Find the line where CSV data starts (after headers)
        data_start_line = 0
        for i, line in enumerate(lines):
            if line.startswith('LAT,LON,YEAR,DOY,ALLSKY_SFC_SW_DWN'):
                data_start_line = i
                break

        # Read CSV starting from the data line
        try:
            self.data = pd.read_csv('/content/POWER_Regional_Daily_20250716_20250723.csv', skiprows=data_start_line, sep=',')
        except Exception as e:
            print(f"Error with comma separator: {e}")
            # Try with different encodings and parameters
            try:
                self.data = pd.read_csv(csv_file_path, skiprows=data_start_line,
                                      sep=',', encoding='utf-8', engine='python')
            except Exception as e2:
                print(f"Error with utf-8: {e2}")
                # Last resort - read manually
                self.data = self._manual_csv_parse(lines[data_start_line:])

        # Clean column names (remove any whitespace)
        self.data.columns = self.data.columns.str.strip()

        # Convert data types
        numeric_columns = ['LAT', 'LON', 'YEAR', 'DOY', 'ALLSKY_SFC_SW_DWN']
        for col in numeric_columns:
            if col in self.data.columns:
                self.data[col] = pd.to_numeric(self.data[col], errors='coerce')

        # Remove missing data (-999 values)
        self.data = self.data[self.data['ALLSKY_SFC_SW_DWN'] != -999.0]
        self.data = self.data.dropna()

        # Convert DOY to actual date for better understanding
        if not self.data.empty:
            self.data['DATE'] = pd.to_datetime(self.data['YEAR'].astype(str) +
                                            self.data['DOY'].astype(str),
                                            format='%Y%j')

        print(f"Data loaded successfully. Shape: {self.data.shape}")
        if not self.data.empty:
            print(f"Date range: {self.data['DATE'].min()} to {self.data['DATE'].max()}")

    def _manual_csv_parse(self, lines):
        """
        Manual CSV parsing as fallback method
        """
        print("Using manual CSV parsing...")
        data_rows = []

        for line in lines:
            line = line.strip()
            if line and not line.startswith('-') and not line.startswith('NASA'):
                parts = line.split(',')
                if len(parts) == 5:
                    try:
                        row = [float(part.strip()) if part.strip() != '' else np.nan for part in parts]
                        data_rows.append(row)
                    except ValueError:
                        if parts[0] == 'LAT':  # Header row
                            continue
                        print(f"Skipping problematic line: {line}")

        if data_rows:
            df = pd.DataFrame(data_rows, columns=['LAT', 'LON', 'YEAR', 'DOY', 'ALLSKY_SFC_SW_DWN'])
            return df
        else:
            raise ValueError("No valid data rows found")

    def calculate_site_metrics(self):
        """
        Calculate average and peak irradiance for each site
        """
        print("Calculating site metrics...")

        self.site_summary = self.data.groupby(['LAT', 'LON']).agg({
            'ALLSKY_SFC_SW_DWN': ['mean', 'max', 'min', 'std', 'count']
        })

        # Flatten column names
        self.site_summary.columns = ['avg_irradiance_mj', 'peak_irradiance_mj',
                                    'min_irradiance_mj', 'std_irradiance_mj', 'data_points']

        # Round the values properly
        for col in ['avg_irradiance_mj', 'peak_irradiance_mj', 'min_irradiance_mj', 'std_irradiance_mj']:
            self.site_summary[col] = self.site_summary[col].round(2)

        # Convert to kWh/m²/day
        self.site_summary['avg_irradiance_kwh'] = (self.site_summary['avg_irradiance_mj'] * self.mj_to_kwh).round(2)
        self.site_summary['peak_irradiance_kwh'] = (self.site_summary['peak_irradiance_mj'] * self.mj_to_kwh).round(2)

        # Reset index to make LAT, LON regular columns
        self.site_summary = self.site_summary.reset_index()

        print(f"Site summary calculated for {len(self.site_summary)} locations")

    def safe_divide_and_round(self, numerator, denominator, default_value=10000, decimal_places=0):
        """
        Safely divide and round, handling edge cases
        """
        # Create a copy to avoid modifying original data
        result = numerator.copy()

        # Handle division by zero and very small numbers
        mask = (denominator <= 0) | (denominator < 1e-10)

        # Perform division where denominator is valid
        valid_mask = ~mask
        result.loc[valid_mask] = numerator.loc[valid_mask] / denominator.loc[valid_mask]

        # Set default value where division is invalid
        result.loc[mask] = default_value

        # Handle inf and nan values
        result = result.replace([np.inf, -np.inf], default_value)
        result = result.fillna(default_value)

        # Round and convert to int if decimal_places is 0
        if decimal_places == 0:
            result = result.round().astype(int)
        else:
            result = result.round(decimal_places)

        return result

    def calculate_solar_potential(self):
        """
        Calculate solar panel requirements and energy yield with updated specifications
        """
        print("Calculating solar potential with 23% efficiency and 320W panels...")

        # Ensure no zero or negative irradiance values
        self.site_summary['avg_irradiance_kwh'] = np.maximum(self.site_summary['avg_irradiance_kwh'], 0.01)

        # Daily energy output per panel (kWh/day/panel)
        self.site_summary['daily_output_per_panel'] = (
            self.site_summary['avg_irradiance_kwh'] *
            self.panel_area *
            self.panel_efficiency
        ).round(3)

        # Ensure minimum daily output to avoid division issues
        self.site_summary['daily_output_per_panel'] = np.maximum(self.site_summary['daily_output_per_panel'], 0.001)

        # Alternative calculation using panel rating (for validation)
        self.site_summary['daily_output_per_panel_rated'] = (
            (self.site_summary['avg_irradiance_kwh'] / 1.0) *  # Normalize to STC
            (self.panel_rating / 1000) *  # Convert watts to kW
            (self.panel_efficiency / 0.20)  # Efficiency correction factor
        ).round(3)

        # Annual energy output per panel (kWh/year/panel)
        self.site_summary['annual_output_per_panel'] = (
            self.site_summary['daily_output_per_panel'] * 365
        ).round(0).astype(int)

        # Number of panels needed for 1 MW daily output
        target_daily_output = 1000  # 1 MW = 1000 kW
        self.site_summary['panels_for_1MW'] = self.safe_divide_and_round(
            pd.Series([target_daily_output] * len(self.site_summary)),
            self.site_summary['daily_output_per_panel'],
            default_value=10000,
            decimal_places=0
        )

        # Number of 320W panels for 1MW nameplate capacity
        panels_nameplate = int(1000000 / self.panel_rating)  # 1MW / 320W per panel = 3125
        self.site_summary['panels_for_1MW_nameplate'] = panels_nameplate

        # Land area required (assuming 4 m² per panel including spacing)
        self.site_summary['land_area_hectares_1MW'] = (
            self.site_summary['panels_for_1MW'] * 4 / 10000
        ).round(2)

        # Land area for nameplate 1MW installation
        self.site_summary['land_area_hectares_1MW_nameplate'] = round(panels_nameplate * 4 / 10000, 2)

        # Calculate capacity factor (actual output vs nameplate capacity)
        nameplate_daily_output = (self.panel_rating / 1000) * 24  # kWh/day at full capacity
        self.site_summary['capacity_factor'] = self.safe_divide_and_round(
            self.site_summary['daily_output_per_panel'],
            pd.Series([nameplate_daily_output] * len(self.site_summary)),
            default_value=0,
            decimal_places=3
        )

        # Daily energy yield per MW nameplate capacity
        self.site_summary['daily_yield_per_MW_nameplate'] = (
            self.site_summary['daily_output_per_panel'] * panels_nameplate
        ).round(2)

        print("Solar potential calculations completed successfully")

    def rank_sites(self):
        """
        Rank sites by solar potential with updated metrics
        """
        # Ensure all required columns exist and have valid values
        required_cols = ['avg_irradiance_kwh', 'capacity_factor', 'peak_irradiance_kwh', 'std_irradiance_mj']
        for col in required_cols:
            if col not in self.site_summary.columns:
                print(f"Warning: Column {col} not found. Setting to default values.")
                self.site_summary[col] = 0

        # Fill any remaining NaN values
        self.site_summary['avg_irradiance_kwh'] = self.site_summary['avg_irradiance_kwh'].fillna(0)
        self.site_summary['capacity_factor'] = self.site_summary['capacity_factor'].fillna(0)
        self.site_summary['peak_irradiance_kwh'] = self.site_summary['peak_irradiance_kwh'].fillna(0)
        self.site_summary['std_irradiance_mj'] = self.site_summary['std_irradiance_mj'].fillna(1)

        # Ensure std_irradiance_mj is not zero to avoid division by zero
        self.site_summary['std_irradiance_mj'] = np.maximum(self.site_summary['std_irradiance_mj'], 0.1)

        # Create composite score (weighted average of metrics)
        self.site_summary['solar_score'] = (
            0.4 * self.site_summary['avg_irradiance_kwh'] +
            0.3 * self.site_summary['capacity_factor'] * 100 +  # Weight capacity factor highly
            0.2 * self.site_summary['peak_irradiance_kwh'] +
            0.1 * (1 / self.site_summary['std_irradiance_mj'])  # Lower std is better
        ).round(3)

        # Rank sites
        self.site_summary['rank'] = self.site_summary['solar_score'].rank(ascending=False, method='dense').astype(int)
        self.site_summary = self.site_summary.sort_values('rank')

        print("Sites ranked by solar potential")

    def train_prediction_model(self, model_type='random_forest'):
        """
        Train ML model to predict irradiance from coordinates
        """
        print(f"Training {model_type} model...")

        # Prepare features and target
        X = self.site_summary[['LAT', 'LON']].copy()
        y = self.site_summary['avg_irradiance_mj'].copy()

        # Remove any invalid values
        valid_mask = ~(X.isna().any(axis=1) | y.isna() | np.isinf(y))
        X = X[valid_mask]
        y = y[valid_mask]

        if len(X) < 3:
            print("Not enough valid data points for model training")
            return None, None

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Choose model
        if model_type == 'linear':
            self.model = LinearRegression()
        else:
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)

        # Train model
        self.model.fit(X_train, y_train)

        # Evaluate model
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Model Performance:")
        print(f"  R² Score: {r2:.3f}")
        print(f"  RMSE: {np.sqrt(mse):.3f} MJ/m²/day")

        return r2, np.sqrt(mse)

    def predict_grid(self, lat_range, lon_range, resolution=0.1):
        """
        Predict irradiance for a grid of points
        """
        if self.model is None:
            print("Please train a model first!")
            return None

        # Create prediction grid
        lat_grid = np.arange(lat_range[0], lat_range[1], resolution)
        lon_grid = np.arange(lon_range[0], lon_range[1], resolution)

        lat_mesh, lon_mesh = np.meshgrid(lat_grid, lon_grid)
        grid_points = np.column_stack([lat_mesh.ravel(), lon_mesh.ravel()])

        # Predict
        predictions = self.model.predict(grid_points)

        # Reshape predictions
        pred_grid = predictions.reshape(lat_mesh.shape)

        return lat_mesh, lon_mesh, pred_grid

    def visualize_results(self):
        """
        Create comprehensive visualizations with updated metrics
        """
        try:
            fig, axes = plt.subplots(2, 3, figsize=(20, 12))

            # 1. Site locations with irradiance
            ax1 = axes[0, 0]
            scatter = ax1.scatter(self.site_summary['LON'], self.site_summary['LAT'],
                                c=self.site_summary['avg_irradiance_kwh'],
                                cmap='YlOrRd', s=100, alpha=0.7)
            ax1.set_xlabel('Longitude')
            ax1.set_ylabel('Latitude')
            ax1.set_title('Average Solar Irradiance by Location')
            plt.colorbar(scatter, ax=ax1, label='Irradiance (kWh/m²/day)')

            # 2. Top 10 sites ranking
            ax2 = axes[0, 1]
            top_10 = self.site_summary.head(min(10, len(self.site_summary)))
            bars = ax2.barh(range(len(top_10)), top_10['avg_irradiance_kwh'])
            ax2.set_yticks(range(len(top_10)))
            ax2.set_yticklabels([f"({lat:.1f}, {lon:.1f})"
                                for lat, lon in zip(top_10['LAT'], top_10['LON'])])
            ax2.set_xlabel('Average Irradiance (kWh/m²/day)')
            ax2.set_title('Top 10 Solar Sites')
            ax2.invert_yaxis()

            # 3. Capacity factor distribution
            ax3 = axes[0, 2]
            valid_cf = self.site_summary['capacity_factor'][self.site_summary['capacity_factor'] > 0]
            if len(valid_cf) > 0:
                ax3.hist(valid_cf, bins=15, alpha=0.7, color='green')
            ax3.set_xlabel('Capacity Factor')
            ax3.set_ylabel('Number of Sites')
            ax3.set_title('Distribution of Capacity Factors (320W panels)')

            # 4. Daily output per panel
            ax4 = axes[1, 0]
            ax4.scatter(self.site_summary['LAT'], self.site_summary['daily_output_per_panel'],
                       alpha=0.7, c='blue')
            ax4.set_xlabel('Latitude')
            ax4.set_ylabel('Daily Output per 320W Panel (kWh/day)')
            ax4.set_title('Solar Output vs Latitude (23% Efficiency)')

            # 5. Land requirement for 1MW
            ax5 = axes[1, 1]
            valid_land = self.site_summary[self.site_summary['land_area_hectares_1MW_nameplate'] < 1000]  # Filter outliers
            ax5.scatter(valid_land['avg_irradiance_kwh'],
                       valid_land['land_area_hectares_1MW_nameplate'],
                       alpha=0.7, c='red')
            ax5.set_xlabel('Average Irradiance (kWh/m²/day)')
            ax5.set_ylabel('Land Area for 1MW (hectares)')
            ax5.set_title('Land Requirement vs Irradiance')

            # 6. Daily yield comparison
            ax6 = axes[1, 2]
            top_10_yield = self.site_summary.head(min(10, len(self.site_summary)))
            ax6.bar(range(len(top_10_yield)), top_10_yield['daily_yield_per_MW_nameplate'])
            ax6.set_xticks(range(len(top_10_yield)))
            ax6.set_xticklabels([f"Site {i+1}" for i in range(len(top_10_yield))], rotation=45)
            ax6.set_ylabel('Daily Yield per MW (kWh/day)')
            ax6.set_title('Top 10 Sites - Daily Energy Yield per MW')

            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Error creating visualizations: {e}")
            print("Continuing with report generation...")

    def generate_report(self):
        """
        Generate comprehensive analysis report with updated specifications
        """
        print("\n" + "="*70)
        print("SOLAR PLANT SITE SELECTION ANALYSIS REPORT")
        print("Updated Panel Specifications: 23% Efficiency, 320W Rating")
        print("="*70)

        print(f"\nPANEL SPECIFICATIONS:")
        print(f"  Panel Efficiency: {self.panel_efficiency*100}%")
        print(f"  Panel Rating: {self.panel_rating}W")
        print(f"  Panel Area: {self.panel_area} m²")

        if len(self.site_summary) == 0:
            print("\nNo data available for analysis.")
            return

        print(f"\nDATASET OVERVIEW:")
        print(f"  Total locations analyzed: {len(self.site_summary)}")
        print(f"  Latitude range: {self.site_summary['LAT'].min():.1f}° to {self.site_summary['LAT'].max():.1f}°")
        print(f"  Longitude range: {self.site_summary['LON'].min():.1f}° to {self.site_summary['LON'].max():.1f}°")

        print(f"\nIRRADIANCE STATISTICS:")
        print(f"  Average irradiance: {self.site_summary['avg_irradiance_kwh'].mean():.2f} kWh/m²/day")
        print(f"  Maximum irradiance: {self.site_summary['avg_irradiance_kwh'].max():.2f} kWh/m²/day")
        print(f"  Minimum irradiance: {self.site_summary['avg_irradiance_kwh'].min():.2f} kWh/m²/day")

        print(f"\nCAPACITY FACTOR STATISTICS:")
        valid_cf = self.site_summary['capacity_factor'][self.site_summary['capacity_factor'] > 0]
        if len(valid_cf) > 0:
            print(f"  Average capacity factor: {valid_cf.mean():.3f} ({valid_cf.mean()*100:.1f}%)")
            print(f"  Maximum capacity factor: {valid_cf.max():.3f} ({valid_cf.max()*100:.1f}%)")
            print(f"  Minimum capacity factor: {valid_cf.min():.3f} ({valid_cf.min()*100:.1f}%)")

        print(f"\nTOP 5 RECOMMENDED SITES:")
        top_5 = self.site_summary.head(min(5, len(self.site_summary)))
        for i, (_, site) in enumerate(top_5.iterrows(), 1):
            print(f"  {i}. Location ({site['LAT']:.1f}°, {site['LON']:.1f}°)")
            print(f"     Average irradiance: {site['avg_irradiance_kwh']:.2f} kWh/m²/day")
            print(f"     Daily output per 320W panel: {site['daily_output_per_panel']:.2f} kWh/day")
            print(f"     Annual output per panel: {site['annual_output_per_panel']:.0f} kWh/year")
            print(f"     Capacity factor: {site['capacity_factor']:.3f} ({site['capacity_factor']*100:.1f}%)")
            print(f"     Panels for 1MW nameplate: {site['panels_for_1MW_nameplate']}")
            print(f"     Daily yield per MW: {site['daily_yield_per_MW_nameplate']:.0f} kWh/day")
            print(f"     Land required: {site['land_area_hectares_1MW_nameplate']:.1f} hectares")
            print()

        print(f"SOLAR POTENTIAL SUMMARY (320W Panels, 23% Efficiency):")
        print(f"  Best site daily output: {self.site_summary['daily_output_per_panel'].max():.2f} kWh/panel/day")
        print(f"  Best site annual output: {self.site_summary['annual_output_per_panel'].max():.0f} kWh/panel/year")
        if len(valid_cf) > 0:
            print(f"  Best site capacity factor: {valid_cf.max():.3f} ({valid_cf.max()*100:.1f}%)")
        print(f"  Standard 1MW installation: {self.site_summary['panels_for_1MW_nameplate'].iloc[0]} panels")
        print(f"  Best site daily yield per MW: {self.site_summary['daily_yield_per_MW_nameplate'].max():.0f} kWh/day")
        print(f"  Minimum land for 1MW: {self.site_summary['land_area_hectares_1MW_nameplate'].min():.1f} hectares")

    def save_results(self, filename='solar_analysis_results_320W_23pct.csv'):
        """
        Save analysis results to CSV
        """
        try:
            self.site_summary.to_csv(filename, index=False)
            print(f"Results saved to {filename}")
        except Exception as e:
            print(f"Error saving results: {e}")

# Alternative main execution that handles file reading issues
def main():
    try:
        # Initialize analyzer with updated specifications
        analyzer = SolarPlantAnalyzer('POWER_Regional_Daily_20250716_20250723.csv')

        # Check if data was loaded successfully
        if analyzer.data is None or analyzer.data.empty:
            print("No data loaded. Please check the file path and format.")
            return

        print(f"Successfully loaded {len(analyzer.data)} data points")
        print(f"Using 320W panels with 23% efficiency")
        print("\nFirst few rows of data:")
        print(analyzer.data.head())

        # Perform analysis
        analyzer.calculate_site_metrics()
        analyzer.calculate_solar_potential()
        analyzer.rank_sites()

        # Train prediction model
        if len(analyzer.site_summary) > 3:  # Need minimum data for ML
            analyzer.train_prediction_model('random_forest')

        # Generate visualizations
        analyzer.visualize_results()

        # Generate report
        analyzer.generate_report()

        # Save results
        analyzer.save_results()

        # Example: Predict for new coordinates (if model was trained)
        if analyzer.model is not None:
            print("\nEXAMPLE PREDICTION FOR NEW LOCATION:")
            new_location = [[20.0, 82.0]]  # Example coordinates
            try:
                predicted_irradiance = analyzer.model.predict(new_location)[0]
                predicted_kwh = predicted_irradiance * 0.2778
                predicted_daily_output = predicted_kwh * analyzer.panel_area * analyzer.panel_efficiency

                print(f"Predicted irradiance at (20.0°, 82.0°): {predicted_irradiance:.2f} MJ/m²/day")
                print(f"Equivalent: {predicted_kwh:.2f} kWh/m²/day")
                print(f"Expected daily output per 320W panel: {predicted_daily_output:.2f} kWh/day")
            except Exception as e:
                print(f"Error making prediction: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Let's try to debug the file structure...")

        # Debug file structure
        try:
            with open('POWER_Regional_Daily_20250716_20250723.csv', 'r') as f:
                lines = f.readlines()[:15]  # Read first 15 lines
                print("\nFirst 15 lines of the file:")
                for i, line in enumerate(lines):
                    print(f"Line {i+1}: {repr(line)}")
        except Exception as file_error:
            print(f"Could not read file: {file_error}")
            print("Please make sure the file 'POWER_Regional_Daily_20250716_20250723.csv' exists in the current directory.")

if __name__ == "__main__":
    main()

Loading NASA POWER dataset...
An error occurred: [Errno 2] No such file or directory: 'POWER_Regional_Daily_20250716_20250723.csv'
Let's try to debug the file structure...
Could not read file: [Errno 2] No such file or directory: 'POWER_Regional_Daily_20250716_20250723.csv'
Please make sure the file 'POWER_Regional_Daily_20250716_20250723.csv' exists in the current directory.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings('ignore')

class SolarPlantAnalyzer:
    def __init__(self, csv_file_path):
        """
        Initialize the Solar Plant Analyzer with the dataset
        """
        self.data = None
        self.site_summary = None
        self.model = None
        self.csv_file_path = csv_file_path
        self.load_data(csv_file_path)

        # Updated solar panel specifications
        self.panel_efficiency = 0.23  # 23% efficiency
        self.panel_rating = 320  # 320 watts per panel
        self.panel_area = 2.0  # 2 m² per panel (typical for 320W panel)
        self.mj_to_kwh = 0.2778  # Conversion factor

    def load_data(self, csv_file_path):
        """
        Load and clean the NASA POWER dataset with MO,DY format
        """
        print("Loading NASA POWER dataset...")
        print(f"Looking for file: {csv_file_path}")

        # Read the file to find where actual data starts
        try:
            with open(csv_file_path, 'r') as file:
                lines = file.readlines()
        except FileNotFoundError:
            print(f"Error: File '{csv_file_path}' not found!")
            print("Please make sure the file exists in the correct location.")
            return

        # Find the line where CSV data starts (after headers)
        data_start_line = 0
        for i, line in enumerate(lines):
            if 'LAT,LON,YEAR,MO,DY,ALLSKY_SFC_SW_DWN' in line:
                data_start_line = i
                break
            elif line.startswith('LAT,LON,YEAR,MO,DY'):
                data_start_line = i
                break

        print(f"Data starts at line: {data_start_line + 1}")

        # Read CSV starting from the data line
        try:
            self.data = pd.read_csv(csv_file_path, skiprows=data_start_line, sep=',')
            print("Successfully loaded data with pandas read_csv")
        except Exception as e:
            print(f"Error with comma separator: {e}")
            # Try with different encodings and parameters
            try:
                self.data = pd.read_csv(csv_file_path, skiprows=data_start_line,
                                      sep=',', encoding='utf-8', engine='python')
                print("Successfully loaded data with utf-8 encoding")
            except Exception as e2:
                print(f"Error with utf-8: {e2}")
                # Last resort - read manually
                print("Attempting manual CSV parsing...")
                self.data = self._manual_csv_parse(lines[data_start_line:])

        if self.data is None or self.data.empty:
            print("Failed to load any data!")
            return

        # Clean column names (remove any whitespace)
        self.data.columns = self.data.columns.str.strip()

        print(f"Columns found: {list(self.data.columns)}")

        # Convert data types - updated for MO,DY format
        numeric_columns = ['LAT', 'LON', 'YEAR', 'MO', 'DY', 'ALLSKY_SFC_SW_DWN']
        for col in numeric_columns:
            if col in self.data.columns:
                self.data[col] = pd.to_numeric(self.data[col], errors='coerce')

        # Remove missing data (-999 values)
        self.data = self.data[self.data['ALLSKY_SFC_SW_DWN'] != -999.0]
        self.data = self.data.dropna()

        # Convert MO,DY to actual date for better understanding
        if not self.data.empty:
            # Create date from YEAR, MO, DY
            self.data['DATE'] = pd.to_datetime(
                self.data[['YEAR', 'MO', 'DY']].rename(columns={'MO': 'month', 'DY': 'day', 'YEAR': 'year'}),
                errors='coerce'
            )

            # Calculate DOY for compatibility with other parts of code if needed
            self.data['DOY'] = self.data['DATE'].dt.dayofyear

        print(f"Data loaded successfully. Shape: {self.data.shape}")
        if not self.data.empty:
            print(f"Date range: {self.data['DATE'].min()} to {self.data['DATE'].max()}")
            print(f"First few rows:")
            print(self.data.head())

    def _manual_csv_parse(self, lines):
        """
        Manual CSV parsing as fallback method for MO,DY format
        """
        print("Using manual CSV parsing...")
        data_rows = []

        for line in lines:
            line = line.strip()
            if line and not line.startswith('-') and not line.startswith('NASA'):
                parts = line.split(',')
                if len(parts) == 6:  # Updated for 6 columns: LAT,LON,YEAR,MO,DY,ALLSKY_SFC_SW_DWN
                    try:
                        row = [float(part.strip()) if part.strip() != '' else np.nan for part in parts]
                        data_rows.append(row)
                    except ValueError:
                        if parts[0] == 'LAT':  # Header row
                            continue
                        print(f"Skipping problematic line: {line}")

        if data_rows:
            df = pd.DataFrame(data_rows, columns=['LAT', 'LON', 'YEAR', 'MO', 'DY', 'ALLSKY_SFC_SW_DWN'])
            return df
        else:
            raise ValueError("No valid data rows found")

    def calculate_site_metrics(self):
        """
        Calculate average and peak irradiance for each site
        """
        print("Calculating site metrics...")

        if self.data is None or self.data.empty:
            print("No data available for site metrics calculation")
            return

        self.site_summary = self.data.groupby(['LAT', 'LON']).agg({
            'ALLSKY_SFC_SW_DWN': ['mean', 'max', 'min', 'std', 'count']
        })

        # Flatten column names
        self.site_summary.columns = ['avg_irradiance_mj', 'peak_irradiance_mj',
                                    'min_irradiance_mj', 'std_irradiance_mj', 'data_points']

        # Round the values properly
        for col in ['avg_irradiance_mj', 'peak_irradiance_mj', 'min_irradiance_mj', 'std_irradiance_mj']:
            self.site_summary[col] = self.site_summary[col].round(2)

        # Convert to kWh/m²/day
        self.site_summary['avg_irradiance_kwh'] = (self.site_summary['avg_irradiance_mj'] * self.mj_to_kwh).round(2)
        self.site_summary['peak_irradiance_kwh'] = (self.site_summary['peak_irradiance_mj'] * self.mj_to_kwh).round(2)

        # Reset index to make LAT, LON regular columns
        self.site_summary = self.site_summary.reset_index()

        print(f"Site summary calculated for {len(self.site_summary)} locations")

    def safe_divide_and_round(self, numerator, denominator, default_value=10000, decimal_places=0):
        """
        Safely divide and round, handling edge cases
        """
        # Create a copy to avoid modifying original data
        result = numerator.copy()

        # Handle division by zero and very small numbers
        mask = (denominator <= 0) | (denominator < 1e-10)

        # Perform division where denominator is valid
        valid_mask = ~mask
        result.loc[valid_mask] = numerator.loc[valid_mask] / denominator.loc[valid_mask]

        # Set default value where division is invalid
        result.loc[mask] = default_value

        # Handle inf and nan values
        result = result.replace([np.inf, -np.inf], default_value)
        result = result.fillna(default_value)

        # Round and convert to int if decimal_places is 0
        if decimal_places == 0:
            result = result.round().astype(int)
        else:
            result = result.round(decimal_places)

        return result

    def calculate_solar_potential(self):
        """
        Calculate solar panel requirements and energy yield with updated specifications
        """
        print("Calculating solar potential with 23% efficiency and 320W panels...")

        if self.site_summary is None or self.site_summary.empty:
            print("No site summary data available for solar potential calculation")
            return

        # Ensure no zero or negative irradiance values
        self.site_summary['avg_irradiance_kwh'] = np.maximum(self.site_summary['avg_irradiance_kwh'], 0.01)

        # Daily energy output per panel (kWh/day/panel)
        self.site_summary['daily_output_per_panel'] = (
            self.site_summary['avg_irradiance_kwh'] *
            self.panel_area *
            self.panel_efficiency
        ).round(3)

        # Ensure minimum daily output to avoid division issues
        self.site_summary['daily_output_per_panel'] = np.maximum(self.site_summary['daily_output_per_panel'], 0.001)

        # Alternative calculation using panel rating (for validation)
        self.site_summary['daily_output_per_panel_rated'] = (
            (self.site_summary['avg_irradiance_kwh'] / 1.0) *  # Normalize to STC
            (self.panel_rating / 1000) *  # Convert watts to kW
            (self.panel_efficiency / 0.20)  # Efficiency correction factor
        ).round(3)

        # Annual energy output per panel (kWh/year/panel)
        self.site_summary['annual_output_per_panel'] = (
            self.site_summary['daily_output_per_panel'] * 365
        ).round(0).astype(int)

        # Number of panels needed for 1 MW daily output
        target_daily_output = 1000  # 1 MW = 1000 kW
        self.site_summary['panels_for_1MW'] = self.safe_divide_and_round(
            pd.Series([target_daily_output] * len(self.site_summary)),
            self.site_summary['daily_output_per_panel'],
            default_value=10000,
            decimal_places=0
        )

        # Number of 320W panels for 1MW nameplate capacity
        panels_nameplate = int(1000000 / self.panel_rating)  # 1MW / 320W per panel = 3125
        self.site_summary['panels_for_1MW_nameplate'] = panels_nameplate

        # Land area required (assuming 4 m² per panel including spacing)
        self.site_summary['land_area_hectares_1MW'] = (
            self.site_summary['panels_for_1MW'] * 4 / 10000
        ).round(2)

        # Land area for nameplate 1MW installation
        self.site_summary['land_area_hectares_1MW_nameplate'] = round(panels_nameplate * 4 / 10000, 2)

        # Calculate capacity factor (actual output vs nameplate capacity)
        nameplate_daily_output = (self.panel_rating / 1000) * 24  # kWh/day at full capacity
        self.site_summary['capacity_factor'] = self.safe_divide_and_round(
            self.site_summary['daily_output_per_panel'],
            pd.Series([nameplate_daily_output] * len(self.site_summary)),
            default_value=0,
            decimal_places=3
        )

        # Daily energy yield per MW nameplate capacity
        self.site_summary['daily_yield_per_MW_nameplate'] = (
            self.site_summary['daily_output_per_panel'] * panels_nameplate
        ).round(2)

        print("Solar potential calculations completed successfully")

    def rank_sites(self):
        """
        Rank sites by solar potential with updated metrics
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No site summary data available for ranking")
            return

        # Ensure all required columns exist and have valid values
        required_cols = ['avg_irradiance_kwh', 'capacity_factor', 'peak_irradiance_kwh', 'std_irradiance_mj']
        for col in required_cols:
            if col not in self.site_summary.columns:
                print(f"Warning: Column {col} not found. Setting to default values.")
                self.site_summary[col] = 0

        # Fill any remaining NaN values
        self.site_summary['avg_irradiance_kwh'] = self.site_summary['avg_irradiance_kwh'].fillna(0)
        self.site_summary['capacity_factor'] = self.site_summary['capacity_factor'].fillna(0)
        self.site_summary['peak_irradiance_kwh'] = self.site_summary['peak_irradiance_kwh'].fillna(0)
        self.site_summary['std_irradiance_mj'] = self.site_summary['std_irradiance_mj'].fillna(1)

        # Ensure std_irradiance_mj is not zero to avoid division by zero
        self.site_summary['std_irradiance_mj'] = np.maximum(self.site_summary['std_irradiance_mj'], 0.1)

        # Create composite score (weighted average of metrics)
        self.site_summary['solar_score'] = (
            0.4 * self.site_summary['avg_irradiance_kwh'] +
            0.3 * self.site_summary['capacity_factor'] * 100 +  # Weight capacity factor highly
            0.2 * self.site_summary['peak_irradiance_kwh'] +
            0.1 * (1 / self.site_summary['std_irradiance_mj'])  # Lower std is better
        ).round(3)

        # Rank sites
        self.site_summary['solar_score'] = self.site_summary['solar_score'].rank(ascending=False, method='dense').astype(int)
        self.site_summary = self.site_summary.sort_values('solar_score')

        print("Sites ranked by solar potential")

    def train_prediction_model(self, model_type='random_forest'):
        """
        Train ML model to predict irradiance from coordinates
        """
        print(f"Training {model_type} model...")

        if self.site_summary is None or len(self.site_summary) < 3:
            print("Not enough data for model training")
            return None, None

        # Prepare features and target
        X = self.site_summary[['LAT', 'LON']].copy()
        y = self.site_summary['avg_irradiance_mj'].copy()

        # Remove any invalid values
        valid_mask = ~(X.isna().any(axis=1) | y.isna() | np.isinf(y))
        X = X[valid_mask]
        y = y[valid_mask]

        if len(X) < 3:
            print("Not enough valid data points for model training")
            return None, None

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Choose model
        if model_type == 'linear':
            self.model = LinearRegression()
        else:
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)

        # Train model
        self.model.fit(X_train, y_train)

        # Evaluate model
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Model Performance:")
        print(f"  R² Score: {r2:.3f}")
        print(f"  RMSE: {np.sqrt(mse):.3f} MJ/m²/day")

        return r2, np.sqrt(mse)

    def predict_grid(self, lat_range, lon_range, resolution=0.1):
        """
        Predict irradiance for a grid of points
        """
        if self.model is None:
            print("Please train a model first!")
            return None

        # Create prediction grid
        lat_grid = np.arange(lat_range[0], lat_range[1], resolution)
        lon_grid = np.arange(lon_range[0], lon_range[1], resolution)

        lat_mesh, lon_mesh = np.meshgrid(lat_grid, lon_grid)
        grid_points = np.column_stack([lat_mesh.ravel(), lon_mesh.ravel()])

        # Predict
        predictions = self.model.predict(grid_points)

        # Reshape predictions
        pred_grid = predictions.reshape(lat_mesh.shape)

        return lat_mesh, lon_mesh, pred_grid

    def visualize_results(self):
        """
        Create comprehensive visualizations with updated metrics
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No data available for visualization")
            return

        try:
            fig, axes = plt.subplots(2, 3, figsize=(20, 12))

            # 1. Site locations with irradiance
            ax1 = axes[0, 0]
            scatter = ax1.scatter(self.site_summary['LON'], self.site_summary['LAT'],
                                c=self.site_summary['avg_irradiance_kwh'],
                                cmap='YlOrRd', s=100, alpha=0.7)
            ax1.set_xlabel('Longitude')
            ax1.set_ylabel('Latitude')
            ax1.set_title('Average Solar Irradiance by Location')
            plt.colorbar(scatter, ax=ax1, label='Irradiance (kWh/m²/day)')

            # 2. Top 10 sites ranking
            ax2 = axes[0, 1]
            top_10 = self.site_summary.head(min(10, len(self.site_summary)))
            bars = ax2.barh(range(len(top_10)), top_10['avg_irradiance_kwh'])
            ax2.set_yticks(range(len(top_10)))
            ax2.set_yticklabels([f"({lat:.1f}, {lon:.1f})"
                                for lat, lon in zip(top_10['LAT'], top_10['LON'])])
            ax2.set_xlabel('Average Irradiance (kWh/m²/day)')
            ax2.set_title('Top 10 Solar Sites')
            ax2.invert_yaxis()

            # 3. Capacity factor distribution
            ax3 = axes[0, 2]
            valid_cf = self.site_summary['capacity_factor'][self.site_summary['capacity_factor'] > 0]
            if len(valid_cf) > 0:
                ax3.hist(valid_cf, bins=15, alpha=0.7, color='green')
            ax3.set_xlabel('Capacity Factor')
            ax3.set_ylabel('Number of Sites')
            ax3.set_title('Distribution of Capacity Factors (320W panels)')

            # 4. Daily output per panel
            ax4 = axes[1, 0]
            ax4.scatter(self.site_summary['LAT'], self.site_summary['daily_output_per_panel'],
                       alpha=0.7, c='blue')
            ax4.set_xlabel('Latitude')
            ax4.set_ylabel('Daily Output per 320W Panel (kWh/day)')
            ax4.set_title('Solar Output vs Latitude (23% Efficiency)')

            # 5. Land requirement for 1MW
            ax5 = axes[1, 1]
            valid_land = self.site_summary[self.site_summary['land_area_hectares_1MW_nameplate'] < 1000]  # Filter outliers
            ax5.scatter(valid_land['avg_irradiance_kwh'],
                       valid_land['land_area_hectares_1MW_nameplate'],
                       alpha=0.7, c='red')
            ax5.set_xlabel('Average Irradiance (kWh/m²/day)')
            ax5.set_ylabel('Land Area for 1MW (hectares)')
            ax5.set_title('Land Requirement vs Irradiance')

            # 6. Daily yield comparison
            ax6 = axes[1, 2]
            top_10_yield = self.site_summary.head(min(10, len(self.site_summary)))
            ax6.bar(range(len(top_10_yield)), top_10_yield['daily_yield_per_MW_nameplate'])
            ax6.set_xticks(range(len(top_10_yield)))
            ax6.set_xticklabels([f"Site {i+1}" for i in range(len(top_10_yield))], rotation=45)
            ax6.set_ylabel('Daily Yield per MW (kWh/day)')
            ax6.set_title('Top 10 Sites - Daily Energy Yield per MW')

            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Error creating visualizations: {e}")
            print("Continuing with report generation...")

    def generate_report(self):
        """
        Generate comprehensive analysis report with updated specifications
        """
        print("\n" + "="*70)
        print("SOLAR PLANT SITE SELECTION ANALYSIS REPORT")
        print("Updated Panel Specifications: 23% Efficiency, 320W Rating")
        print("="*70)

        print(f"\nPANEL SPECIFICATIONS:")
        print(f"  Panel Efficiency: {self.panel_efficiency*100}%")
        print(f"  Panel Rating: {self.panel_rating}W")
        print(f"  Panel Area: {self.panel_area} m²")

        if self.site_summary is None or len(self.site_summary) == 0:
            print("\nNo data available for analysis.")
            return

        print(f"\nDATASET OVERVIEW:")
        print(f"  Total locations analyzed: {len(self.site_summary)}")
        print(f"  Latitude range: {self.site_summary['LAT'].min():.1f}° to {self.site_summary['LAT'].max():.1f}°")
        print(f"  Longitude range: {self.site_summary['LON'].min():.1f}° to {self.site_summary['LON'].max():.1f}°")

        print(f"\nIRRADIANCE STATISTICS:")
        print(f"  Average irradiance: {self.site_summary['avg_irradiance_kwh'].mean():.2f} kWh/m²/day")
        print(f"  Maximum irradiance: {self.site_summary['avg_irradiance_kwh'].max():.2f} kWh/m²/day")
        print(f"  Minimum irradiance: {self.site_summary['avg_irradiance_kwh'].min():.2f} kWh/m²/day")

        print(f"\nCAPACITY FACTOR STATISTICS:")
        valid_cf = self.site_summary['capacity_factor'][self.site_summary['capacity_factor'] > 0]
        if len(valid_cf) > 0:
            print(f"  Average capacity factor: {valid_cf.mean():.3f} ({valid_cf.mean()*100:.1f}%)")
            print(f"  Maximum capacity factor: {valid_cf.max():.3f} ({valid_cf.max()*100:.1f}%)")
            print(f"  Minimum capacity factor: {valid_cf.min():.3f} ({valid_cf.min()*100:.1f}%)")

        print(f"\nTOP 5 RECOMMENDED SITES:")
        top_5 = self.site_summary.head(min(5, len(self.site_summary)))
        for i, (_, site) in enumerate(top_5.iterrows(), 1):
            print(f"  {i}. Location ({site['LAT']:.1f}°, {site['LON']:.1f}°)")
            print(f"     Average irradiance: {site['avg_irradiance_kwh']:.2f} kWh/m²/day")
            print(f"     Daily output per 320W panel: {site['daily_output_per_panel']:.2f} kWh/day")
            print(f"     Annual output per panel: {site['annual_output_per_panel']:.0f} kWh/year")
            print(f"     Capacity factor: {site['capacity_factor']:.3f} ({site['capacity_factor']*100:.1f}%)")
            print(f"     Panels for 1MW nameplate: {site['panels_for_1MW_nameplate']}")
            print(f"     Daily yield per MW: {site['daily_yield_per_MW_nameplate']:.0f} kWh/day")
            print(f"     Land required: {site['land_area_hectares_1MW_nameplate']:.1f} hectares")
            print()

        print(f"SOLAR POTENTIAL SUMMARY (320W Panels, 23% Efficiency):")
        print(f"  Best site daily output: {self.site_summary['daily_output_per_panel'].max():.2f} kWh/panel/day")
        print(f"  Best site annual output: {self.site_summary['annual_output_per_panel'].max():.0f} kWh/panel/year")
        if len(valid_cf) > 0:
            print(f"  Best site capacity factor: {valid_cf.max():.3f} ({valid_cf.max()*100:.1f}%)")
        print(f"  Standard 1MW installation: {self.site_summary['panels_for_1MW_nameplate'].iloc[0]} panels")
        print(f"  Best site daily yield per MW: {self.site_summary['daily_yield_per_MW_nameplate'].max():.0f} kWh/day")
        print(f"  Minimum land for 1MW: {self.site_summary['land_area_hectares_1MW_nameplate'].min():.1f} hectares")

    def save_results(self, filename='solar_analysis_results_320W_23pct.csv'):
        """
        Save analysis results to CSV
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No results to save")
            return

        try:
            self.site_summary.to_csv(filename, index=False)
            print(f"Results saved to {filename}")
        except Exception as e:
            print(f"Error saving results: {e}")

# Main execution function
def main():
    try:
        # Initialize analyzer with the correct file path
        print("Starting Solar Plant Analysis...")
        print("Expected CSV format: LAT,LON,YEAR,MO,DY,ALLSKY_SFC_SW_DWN")

        # Use the correct file path that you uploaded
        csv_file_path = '/content/POWER_Regional_Daily_20250601_20250630 .csv'
        analyzer = SolarPlantAnalyzer(csv_file_path)

        # Check if data was loaded successfully
        if analyzer.data is None or analyzer.data.empty:
            print("No data loaded. Please check the file path and format.")
            return

        print(f"Successfully loaded {len(analyzer.data)} data points")
        print(f"Using 320W panels with 23% efficiency")

        # Perform analysis
        analyzer.calculate_site_metrics()
        analyzer.calculate_solar_potential()
        analyzer.rank_sites()

        # Train prediction model
        if analyzer.site_summary is not None and len(analyzer.site_summary) > 3:  # Need minimum data for ML
            analyzer.train_prediction_model('random_forest')

        # Generate visualizations
        analyzer.visualize_results()

        # Generate report
        analyzer.generate_report()

        # Save results
        analyzer.save_results()

        # Example: Predict for new coordinates (if model was trained)
        if analyzer.model is not None:
            print("\nEXAMPLE PREDICTION FOR NEW LOCATION:")
            new_location = [[20.0, 82.0]]  # Example coordinates
            try:
                predicted_irradiance = analyzer.model.predict(new_location)[0]
                predicted_kwh = predicted_irradiance * 0.2778
                predicted_daily_output = predicted_kwh * analyzer.panel_area * analyzer.panel_efficiency

                print(f"Predicted irradiance at (20.0°, 82.0°): {predicted_irradiance:.2f} MJ/m²/day")
                print(f"Equivalent: {predicted_kwh:.2f} kWh/m²/day")
                print(f"Expected daily output per 320W panel: {predicted_daily_output:.2f} kWh/day")
            except Exception as e:
                print(f"Error making prediction: {e}")

        print("\nAnalysis completed successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Let's try to debug the file structure...")

        # Debug file structure - use the correct file path
        csv_file_path = '/content/POWER_Regional_Daily_20250601_20250630 .csv'
        try:
            with open(csv_file_path, 'r') as f:
                lines = f.readlines()[:20]  # Read first 20 lines
                print("\nFirst 20 lines of the file:")
                for i, line in enumerate(lines):
                    print(f"Line {i+1}: {repr(line)}")
        except Exception as file_error:
            print(f"Could not read file: {file_error}")
            print(f"Please make sure the file '{csv_file_path}' exists in the current directory.")

            # Try alternative file paths
            alternative_paths = [
                'POWER_Regional_Daily_20250601_20250630 .csv',
                './POWER_Regional_Daily_20250601_20250630 .csv',
                'POWER_Regional_Daily_20250601_20250630.csv',  # without space
                '/content/POWER_Regional_Daily_20250601_20250630.csv'  # without space
            ]

            print("\nTrying alternative file paths:")
            for alt_path in alternative_paths:
                try:
                    with open(alt_path, 'r') as f:
                        print(f"✓ Found file at: {alt_path}")
                        lines = f.readlines()[:5]
                        print("First 5 lines:")
                        for i, line in enumerate(lines):
                            print(f"  {i+1}: {line.strip()}")
                        break
                except:
                    print(f"✗ Not found: {alt_path}")

if __name__ == "__main__":
    main()

Starting Solar Plant Analysis...
Expected CSV format: LAT,LON,YEAR,MO,DY,ALLSKY_SFC_SW_DWN
Loading NASA POWER dataset...
Looking for file: /content/POWER_Regional_Daily_20250601_20250630 .csv
Error: File '/content/POWER_Regional_Daily_20250601_20250630 .csv' not found!
Please make sure the file exists in the correct location.
No data loaded. Please check the file path and format.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings('ignore')

class SolarPlantAnalyzer:
    def __init__(self, csv_file_path):
        """
        Initialize the Solar Plant Analyzer with NASA POWER monthly dataset
        """
        self.data = None
        self.site_summary = None
        self.model = None
        self.csv_file_path = csv_file_path
        self.load_data(csv_file_path)

        # Solar panel specifications
        self.panel_efficiency = 0.23  # 23% efficiency
        self.panel_rating = 320  # 320 watts per panel
        self.panel_area = 2.0  # 2 m² per panel (typical for 320W panel)
        self.performance_ratio = 0.85  # System losses factor

    def load_data(self, csv_file_path):
        """
        Load and clean the NASA POWER monthly dataset
        """
        print("Loading NASA POWER monthly dataset...")
        print(f"Looking for file: {csv_file_path}")

        try:
            # Read the file to find where actual data starts
            with open(csv_file_path, 'r') as file:
                lines = file.readlines()

            # Find the header line with PARAMETER,YEAR,LAT,LON...
            data_start_line = 0
            for i, line in enumerate(lines):
                if line.startswith('PARAMETER,YEAR,LAT,LON'):
                    data_start_line = i
                    break

            print(f"Data starts at line: {data_start_line + 1}")

            # Read CSV starting from the data line
            self.data = pd.read_csv(csv_file_path, skiprows=data_start_line)
            print("Successfully loaded data")

        except FileNotFoundError:
            print(f"Error: File '{csv_file_path}' not found!")
            return
        except Exception as e:
            print(f"Error loading data: {e}")
            return

        if self.data is None or self.data.empty:
            print("Failed to load any data!")
            return

        # Clean column names
        self.data.columns = self.data.columns.str.strip()
        print(f"Columns found: {list(self.data.columns)}")

        # Filter for ALLSKY_SFC_SW_DWN parameter only
        self.data = self.data[self.data['PARAMETER'] == 'ALLSKY_SFC_SW_DWN']

        # Convert data types
        numeric_columns = ['YEAR', 'LAT', 'LON', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                          'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANN']

        for col in numeric_columns:
            if col in self.data.columns:
                self.data[col] = pd.to_numeric(self.data[col], errors='coerce')

        # Remove missing data (-999 values)
        month_cols = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                     'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

        for col in month_cols:
            if col in self.data.columns:
                self.data = self.data[self.data[col] != -999.0]

        self.data = self.data.dropna()

        print(f"Data loaded successfully. Shape: {self.data.shape}")
        if not self.data.empty:
            print(f"Year range: {self.data['YEAR'].min()} to {self.data['YEAR'].max()}")
            print(f"Latitude range: {self.data['LAT'].min()}° to {self.data['LAT'].max()}°")
            print(f"Longitude range: {self.data['LON'].min()}° to {self.data['LON'].max()}°")
            print(f"First few rows:")
            print(self.data.head())

    def calculate_site_metrics(self):
        """
        Calculate average and seasonal irradiance metrics for each site
        """
        print("Calculating site metrics...")

        if self.data is None or self.data.empty:
            print("No data available for site metrics calculation")
            return

        # Group by location and calculate statistics
        month_cols = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                     'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

        # Calculate metrics for each site
        site_data = []

        for (lat, lon), group in self.data.groupby(['LAT', 'LON']):
            # Calculate annual averages across all years
            annual_avg = group['ANN'].mean()
            annual_max = group['ANN'].max()
            annual_min = group['ANN'].min()
            annual_std = group['ANN'].std()

            # Calculate monthly averages across all years
            monthly_avgs = {}
            for month in month_cols:
                monthly_avgs[f'avg_{month.lower()}'] = group[month].mean()

            # Calculate seasonal averages
            # Summer: JUN, JUL, AUG
            summer_avg = group[['JUN', 'JUL', 'AUG']].mean().mean()
            # Winter: DEC, JAN, FEB
            winter_avg = group[['DEC', 'JAN', 'FEB']].mean().mean()
            # Monsoon: JUL, AUG, SEP (for India)
            monsoon_avg = group[['JUL', 'AUG', 'SEP']].mean().mean()

            site_info = {
                'LAT': lat,
                'LON': lon,
                'avg_irradiance_kwh': annual_avg,
                'max_irradiance_kwh': annual_max,
                'min_irradiance_kwh': annual_min,
                'std_irradiance_kwh': annual_std if not np.isnan(annual_std) else 0.1,
                'summer_avg_kwh': summer_avg,
                'winter_avg_kwh': winter_avg,
                'monsoon_avg_kwh': monsoon_avg,
                'seasonal_variation': summer_avg - winter_avg,
                'data_points': len(group)
            }

            # Add monthly averages
            site_info.update(monthly_avgs)

            site_data.append(site_info)

        self.site_summary = pd.DataFrame(site_data)

        # Round values
        numeric_cols = [col for col in self.site_summary.columns if col not in ['LAT', 'LON', 'data_points']]
        for col in numeric_cols:
            self.site_summary[col] = self.site_summary[col].round(3)

        print(f"Site summary calculated for {len(self.site_summary)} locations")

    def calculate_solar_potential(self):
        """
        Calculate solar panel requirements and energy yield with updated specifications
        """
        print("Calculating solar potential with 23% efficiency and 320W panels...")

        if self.site_summary is None or self.site_summary.empty:
            print("No site summary data available for solar potential calculation")
            return

        # Ensure no zero or negative irradiance values
        self.site_summary['avg_irradiance_kwh'] = np.maximum(self.site_summary['avg_irradiance_kwh'], 0.01)

        # Daily energy output per panel (kWh/day/panel)
        # Formula: Irradiance (kWh/m²/day) × Panel Area (m²) × Efficiency × Performance Ratio
        self.site_summary['daily_output_per_panel'] = (
            self.site_summary['avg_irradiance_kwh'] *
            self.panel_area *
            self.panel_efficiency *
            self.performance_ratio
        ).round(3)

        # Ensure minimum daily output to avoid division issues
        self.site_summary['daily_output_per_panel'] = np.maximum(self.site_summary['daily_output_per_panel'], 0.001)

        # Seasonal energy output per panel
        self.site_summary['summer_output_per_panel'] = (
            self.site_summary['summer_avg_kwh'] *
            self.panel_area *
            self.panel_efficiency *
            self.performance_ratio
        ).round(3)

        self.site_summary['winter_output_per_panel'] = (
            self.site_summary['winter_avg_kwh'] *
            self.panel_area *
            self.panel_efficiency *
            self.performance_ratio
        ).round(3)

        # Annual energy output per panel (kWh/year/panel)
        self.site_summary['annual_output_per_panel'] = (
            self.site_summary['daily_output_per_panel'] * 365
        ).round(0).astype(int)

        # Number of 320W panels for 1MW nameplate capacity
        panels_nameplate = int(1000000 / self.panel_rating)  # 1MW / 320W per panel = 3125
        self.site_summary['panels_for_1MW_nameplate'] = panels_nameplate

        # Daily energy yield per MW nameplate capacity
        self.site_summary['daily_yield_per_MW_nameplate'] = (
            self.site_summary['daily_output_per_panel'] * panels_nameplate
        ).round(2)

        # Annual energy yield per MW nameplate capacity
        self.site_summary['annual_yield_per_MW_nameplate'] = (
            self.site_summary['annual_output_per_panel'] * panels_nameplate
        ).round(0).astype(int)

        # Calculate capacity factor (actual output vs nameplate capacity)
        nameplate_daily_output = (self.panel_rating / 1000) * 24  # kWh/day at full capacity
        self.site_summary['capacity_factor'] = (
            self.site_summary['daily_output_per_panel'] / nameplate_daily_output
        ).round(4)

        # Land area required (assuming 4 m² per panel including spacing)
        self.site_summary['land_area_hectares_1MW'] = round(panels_nameplate * 4 / 10000, 2)

        # Economic metrics
        # Assuming cost per panel installation: $200
        cost_per_panel = 200
        self.site_summary['installation_cost_1MW'] = panels_nameplate * cost_per_panel

        # Simple payback calculation (assuming $0.1 per kWh)
        electricity_price = 0.1  # $/kWh
        annual_revenue_per_MW = self.site_summary['annual_yield_per_MW_nameplate'] * electricity_price
        self.site_summary['simple_payback_years'] = (
            self.site_summary['installation_cost_1MW'] / annual_revenue_per_MW
        ).round(1)

        print("Solar potential calculations completed successfully")

    def rank_sites(self):
        """
        Rank sites by solar potential with comprehensive scoring
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No site summary data available for ranking")
            return

        # Ensure all required columns exist and have valid values
        required_cols = ['avg_irradiance_kwh', 'capacity_factor', 'std_irradiance_kwh', 'seasonal_variation']
        for col in required_cols:
            if col not in self.site_summary.columns:
                print(f"Warning: Column {col} not found. Setting to default values.")
                self.site_summary[col] = 0

        # Fill any remaining NaN values
        self.site_summary['std_irradiance_kwh'] = self.site_summary['std_irradiance_kwh'].fillna(0.1)
        self.site_summary['std_irradiance_kwh'] = np.maximum(self.site_summary['std_irradiance_kwh'], 0.1)

        # Normalize metrics for scoring (0-100 scale)
        def normalize_score(series, higher_is_better=True):
            if series.max() == series.min():
                return pd.Series([50] * len(series), index=series.index)

            if higher_is_better:
                return ((series - series.min()) / (series.max() - series.min()) * 100).round(2)
            else:
                return ((series.max() - series) / (series.max() - series.min()) * 100).round(2)

        # Calculate individual scores
        self.site_summary['irradiance_score'] = normalize_score(self.site_summary['avg_irradiance_kwh'])
        self.site_summary['capacity_score'] = normalize_score(self.site_summary['capacity_factor'])
        self.site_summary['stability_score'] = normalize_score(self.site_summary['std_irradiance_kwh'], False)
        self.site_summary['seasonal_score'] = normalize_score(abs(self.site_summary['seasonal_variation']), False)

        # Create composite score (weighted average)
        self.site_summary['solar_score'] = (
            0.4 * self.site_summary['irradiance_score'] +
            0.3 * self.site_summary['capacity_score'] +
            0.2 * self.site_summary['stability_score'] +
            0.1 * self.site_summary['seasonal_score']
        ).round(2)

        # Rank sites (1 = best)
        self.site_summary['rank'] = self.site_summary['solar_score'].rank(ascending=False, method='dense').astype(int)
        self.site_summary = self.site_summary.sort_values('rank')

        print("Sites ranked by solar potential")

    def train_prediction_model(self, model_type='random_forest'):
        """
        Train ML model to predict irradiance from coordinates
        """
        print(f"Training {model_type} model for irradiance prediction...")

        if self.site_summary is None or len(self.site_summary) < 3:
            print("Not enough data for model training")
            return None, None

        # Prepare features and target
        X = self.site_summary[['LAT', 'LON']].copy()
        y = self.site_summary['avg_irradiance_kwh'].copy()

        # Remove any invalid values
        valid_mask = ~(X.isna().any(axis=1) | y.isna() | np.isinf(y))
        X = X[valid_mask]
        y = y[valid_mask]

        if len(X) < 3:
            print("Not enough valid data points for model training")
            return None, None

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Choose and train model
        if model_type == 'linear':
            self.model = LinearRegression()
        else:
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)

        self.model.fit(X_train, y_train)

        # Evaluate model
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Model Performance:")
        print(f"  R² Score: {r2:.3f}")
        print(f"  RMSE: {np.sqrt(mse):.3f} kWh/m²/day")

        return r2, np.sqrt(mse)

    def visualize_results(self):
        """
        Create comprehensive visualizations
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No data available for visualization")
            return

        try:
            # Set up the plot style
            plt.style.use('default')
            fig = plt.figure(figsize=(20, 16))

            # 1. Geographic distribution of solar irradiance
            ax1 = plt.subplot(3, 3, 1)
            scatter = ax1.scatter(self.site_summary['LON'], self.site_summary['LAT'],
                                c=self.site_summary['avg_irradiance_kwh'],
                                cmap='YlOrRd', s=150, alpha=0.8, edgecolors='black', linewidth=0.5)
            ax1.set_xlabel('Longitude (°)', fontsize=10)
            ax1.set_ylabel('Latitude (°)', fontsize=10)
            ax1.set_title('Average Solar Irradiance by Location\n(kWh/m²/day)', fontsize=11, fontweight='bold')
            plt.colorbar(scatter, ax=ax1, label='Irradiance (kWh/m²/day)')
            ax1.grid(True, alpha=0.3)

            # 2. Top 10 sites ranking
            ax2 = plt.subplot(3, 3, 2)
            top_10 = self.site_summary.head(min(10, len(self.site_summary)))
            bars = ax2.barh(range(len(top_10)), top_10['avg_irradiance_kwh'], color='orange', alpha=0.8)
            ax2.set_yticks(range(len(top_10)))
            ax2.set_yticklabels([f"Rank {row['rank']}: ({row['LAT']:.1f}°, {row['LON']:.1f}°)"
                                for _, row in top_10.iterrows()], fontsize=9)
            ax2.set_xlabel('Average Irradiance (kWh/m²/day)', fontsize=10)
            ax2.set_title('Top 10 Solar Sites', fontsize=11, fontweight='bold')
            ax2.invert_yaxis()
            ax2.grid(True, alpha=0.3, axis='x')

            # 3. Capacity factor distribution
            ax3 = plt.subplot(3, 3, 3)
            valid_cf = self.site_summary['capacity_factor'][self.site_summary['capacity_factor'] > 0]
            if len(valid_cf) > 0:
                ax3.hist(valid_cf * 100, bins=15, alpha=0.7, color='green', edgecolor='black')
            ax3.set_xlabel('Capacity Factor (%)', fontsize=10)
            ax3.set_ylabel('Number of Sites', fontsize=10)
            ax3.set_title('Distribution of Capacity Factors\n(320W panels, 23% efficiency)', fontsize=11, fontweight='bold')
            ax3.grid(True, alpha=0.3)

            # 4. Seasonal variation
            ax4 = plt.subplot(3, 3, 4)
            ax4.scatter(self.site_summary['winter_avg_kwh'], self.site_summary['summer_avg_kwh'],
                       alpha=0.7, c='blue', s=80, edgecolors='black', linewidth=0.5)
            ax4.set_xlabel('Winter Average (kWh/m²/day)', fontsize=10)
            ax4.set_ylabel('Summer Average (kWh/m²/day)', fontsize=10)
            ax4.set_title('Seasonal Irradiance Variation', fontsize=11, fontweight='bold')
            # Add diagonal line for reference
            min_val = min(self.site_summary['winter_avg_kwh'].min(), self.site_summary['summer_avg_kwh'].min())
            max_val = max(self.site_summary['winter_avg_kwh'].max(), self.site_summary['summer_avg_kwh'].max())
            ax4.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.5, label='Equal seasons')
            ax4.legend()
            ax4.grid(True, alpha=0.3)

            # 5. Daily output per panel vs latitude
            ax5 = plt.subplot(3, 3, 5)
            ax5.scatter(self.site_summary['LAT'], self.site_summary['daily_output_per_panel'],
                       alpha=0.7, c='purple', s=80, edgecolors='black', linewidth=0.5)
            ax5.set_xlabel('Latitude (°)', fontsize=10)
            ax5.set_ylabel('Daily Output per Panel (kWh/day)', fontsize=10)
            ax5.set_title('Solar Output vs Latitude\n(320W panels, 23% efficiency)', fontsize=11, fontweight='bold')
            ax5.grid(True, alpha=0.3)

            # 6. Annual energy yield per MW
            ax6 = plt.subplot(3, 3, 6)
            top_10_yield = self.site_summary.head(min(10, len(self.site_summary)))
            bars = ax6.bar(range(len(top_10_yield)), top_10_yield['annual_yield_per_MW_nameplate'] / 1000,
                          color='red', alpha=0.8)
            ax6.set_xticks(range(len(top_10_yield)))
            ax6.set_xticklabels([f"Rank {row['rank']}" for _, row in top_10_yield.iterrows()], rotation=45)
            ax6.set_ylabel('Annual Yield per MW (GWh/year)', fontsize=10)
            ax6.set_title('Top 10 Sites - Annual Energy Yield per MW', fontsize=11, fontweight='bold')
            ax6.grid(True, alpha=0.3, axis='y')

            # 7. Solar score vs irradiance
            ax7 = plt.subplot(3, 3, 7)
            ax7.scatter(self.site_summary['avg_irradiance_kwh'], self.site_summary['solar_score'],
                       alpha=0.7, c='brown', s=80, edgecolors='black', linewidth=0.5)
            ax7.set_xlabel('Average Irradiance (kWh/m²/day)', fontsize=10)
            ax7.set_ylabel('Solar Score', fontsize=10)
            ax7.set_title('Composite Solar Score vs Irradiance', fontsize=11, fontweight='bold')
            ax7.grid(True, alpha=0.3)

            # 8. Monthly irradiance pattern for top site
            ax8 = plt.subplot(3, 3, 8)
            if not self.site_summary.empty:
                top_site = self.site_summary.iloc[0]
                months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                monthly_values = [top_site[f'avg_{month.lower()}'] for month in
                                ['jan', 'feb', 'mar', 'apr', 'may', 'jun',
                                 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']]
                ax8.plot(months, monthly_values, 'o-', linewidth=2, markersize=6, color='darkgreen')
                ax8.set_ylabel('Irradiance (kWh/m²/day)', fontsize=10)
                ax8.set_title(f'Monthly Pattern - Best Site\n(Rank 1: {top_site["LAT"]:.1f}°, {top_site["LON"]:.1f}°)',
                             fontsize=11, fontweight='bold')
                ax8.tick_params(axis='x', rotation=45)
                ax8.grid(True, alpha=0.3)

            # 9. Payback period analysis
            ax9 = plt.subplot(3, 3, 9)
            if 'simple_payback_years' in self.site_summary.columns:
                valid_payback = self.site_summary[self.site_summary['simple_payback_years'] > 0]
                if len(valid_payback) > 0:
                    ax9.hist(valid_payback['simple_payback_years'], bins=15, alpha=0.7,
                            color='gold', edgecolor='black')
                    ax9.set_xlabel('Simple Payback Period (years)', fontsize=10)
                    ax9.set_ylabel('Number of Sites', fontsize=10)
                    ax9.set_title('Investment Payback Period\n(@$0.1/kWh)', fontsize=11, fontweight='bold')
                    ax9.grid(True, alpha=0.3)

            plt.tight_layout(pad=3.0)
            plt.show()

        except Exception as e:
            print(f"Error creating visualizations: {e}")
            print("Continuing with report generation...")

    def generate_report(self):
        """
        Generate comprehensive analysis report
        """
        print("\n" + "="*80)
        print("SOLAR PLANT SITE SELECTION ANALYSIS REPORT")
        print("Panel Specifications: 23% Efficiency, 320W Rating")
        print("Data Source: NASA POWER Monthly Irradiance Data")
        print("="*80)

        print(f"\nPANEL SPECIFICATIONS:")
        print(f"  Panel Efficiency: {self.panel_efficiency*100}%")
        print(f"  Panel Rating: {self.panel_rating}W")
        print(f"  Panel Area: {self.panel_area} m²")
        print(f"  Performance Ratio: {self.performance_ratio}")

        if self.site_summary is None or len(self.site_summary) == 0:
            print("\nNo data available for analysis.")
            return

        print(f"\nDATASET OVERVIEW:")
        print(f"  Total locations analyzed: {len(self.site_summary)}")
        print(f"  Latitude range: {self.site_summary['LAT'].min():.1f}° to {self.site_summary['LAT'].max():.1f}°")
        print(f"  Longitude range: {self.site_summary['LON'].min():.1f}° to {self.site_summary['LON'].max():.1f}°")
        print(f"  Years covered: {self.data['YEAR'].min()} to {self.data['YEAR'].max()}")

        print(f"\nIRRADIANCE STATISTICS (kWh/m²/day):")
        print(f"  Average irradiance: {self.site_summary['avg_irradiance_kwh'].mean():.2f}")
        print(f"  Maximum irradiance: {self.site_summary['avg_irradiance_kwh'].max():.2f}")
        print(f"  Minimum irradiance: {self.site_summary['avg_irradiance_kwh'].min():.2f}")
        print(f"  Standard deviation: {self.site_summary['avg_irradiance_kwh'].std():.2f}")

        print(f"\nSEASONAL ANALYSIS:")
        print(f"  Average summer irradiance: {self.site_summary['summer_avg_kwh'].mean():.2f} kWh/m²/day")
        print(f"  Average winter irradiance: {self.site_summary['winter_avg_kwh'].mean():.2f} kWh/m²/day")
        print(f"  Average monsoon irradiance: {self.site_summary['monsoon_avg_kwh'].mean():.2f} kWh/m²/day")
        print(f"  Average seasonal variation: {self.site_summary['seasonal_variation'].mean():.2f} kWh/m²/day")

        print(f"\nCAPACITY FACTOR STATISTICS:")
        valid_cf = self.site_summary['capacity_factor'][self.site_summary['capacity_factor'] > 0]
        if len(valid_cf) > 0:
            print(f"  Average capacity factor: {valid_cf.mean():.3f} ({valid_cf.mean()*100:.1f}%)")
            print(f"  Maximum capacity factor: {valid_cf.max():.3f} ({valid_cf.max()*100:.1f}%)")
            print(f"  Minimum capacity factor: {valid_cf.min():.3f} ({valid_cf.min()*100:.1f}%)")

        print(f"\nTOP 5 RECOMMENDED SITES:")
        print("-" * 80)
        top_5 = self.site_summary.head(min(5, len(self.site_summary)))
        for i, (_, site) in enumerate(top_5.iterrows(), 1):
            print(f"  RANK {site['rank']} - Location ({site['LAT']:.1f}°, {site['LON']:.1f}°)")
            print(f"     Average irradiance: {site['avg_irradiance_kwh']:.2f} kWh/m²/day")
            print(f"     Daily output per panel: {site['daily_output_per_panel']:.2f} kWh/day")
            print(f"     Annual output per panel: {site['annual_output_per_panel']:.0f} kWh/year")
            print(f"     Capacity factor: {site['capacity_factor']:.3f} ({site['capacity_factor']*100:.1f}%)")
            print(f"     Daily yield per MW: {site['daily_yield_per_MW_nameplate']:.0f} kWh/day")
            print(f"     Annual yield per MW: {site['annual_yield_per_MW_nameplate']:.0f} kWh/year")
            print(f"     Summer output: {site['summer_output_per_panel']:.2f} kWh/panel/day")
            print(f"     Winter output: {site['winter_output_per_panel']:.2f} kWh/panel/day")
            print(f"     Solar score: {site['solar_score']:.1f}/100")
            if 'simple_payback_years' in site:
                print(f"     Simple payback: {site['simple_payback_years']:.1f} years")
            print(f"     Land required: {site['land_area_hectares_1MW']:.1f} hectares per MW")
            print()

        print(f"SOLAR POTENTIAL SUMMARY:")
        print("-" * 50)
        print(f"  Best site daily output: {self.site_summary['daily_output_per_panel'].max():.2f} kWh/panel/day")
        print(f"  Best site annual output: {self.site_summary['annual_output_per_panel'].max():.0f} kWh/panel/year")
        if len(valid_cf) > 0:
            print(f"  Best site capacity factor: {valid_cf.max():.3f} ({valid_cf.max()*100:.1f}%)")
        print(f"  Standard 1MW installation: {self.site_summary['panels_for_1MW_nameplate'].iloc[0]} panels")
        print(f"  Best site daily yield per MW: {self.site_summary['daily_yield_per_MW_nameplate'].max():.0f} kWh/day")
        print(f"  Best site annual yield per MW: {self.site_summary['annual_yield_per_MW_nameplate'].max():.0f} kWh/year")
        print(f"  Land requirement per MW: {self.site_summary['land_area_hectares_1MW'].iloc[0]:.1f} hectares")

        # Economic analysis
        if 'simple_payback_years' in self.site_summary.columns:
            valid_payback = self.site_summary[self.site_summary['simple_payback_years'] > 0]
            if len(valid_payback) > 0:
                print(f"  Best payback period: {valid_payback['simple_payback_years'].min():.1f} years")
                print(f"  Average payback period: {valid_payback['simple_payback_years'].mean():.1f} years")

        print(f"\nRECOMMENDATIONS:")
        print("-" * 30)
        best_site = self.site_summary.iloc[0]
        print(f"  1. Primary recommendation: Location ({best_site['LAT']:.1f}°, {best_site['LON']:.1f}°)")
        print(f"     - Highest solar score: {best_site['solar_score']:.1f}/100")
        print(f"     - Expected capacity factor: {best_site['capacity_factor']*100:.1f}%")
        print(f"     - Low seasonal variation: {abs(best_site['seasonal_variation']):.2f} kWh/m²/day")

        # Find sites with good monsoon performance
        monsoon_resistant = self.site_summary[self.site_summary['monsoon_avg_kwh'] > self.site_summary['monsoon_avg_kwh'].median()]
        if not monsoon_resistant.empty:
            print(f"  2. Monsoon-resistant sites: {len(monsoon_resistant)} locations with above-median monsoon performance")

        # Find sites with consistent performance
        stable_sites = self.site_summary[self.site_summary['std_irradiance_kwh'] < self.site_summary['std_irradiance_kwh'].median()]
        if not stable_sites.empty:
            print(f"  3. Most stable sites: {len(stable_sites)} locations with low irradiance variation")

    def predict_new_location(self, lat, lon):
        """
        Predict solar potential for a new location
        """
        if self.model is None:
            print("Please train a model first using train_prediction_model()!")
            return None

        try:
            # Make prediction
            predicted_irradiance = self.model.predict([[lat, lon]])[0]

            # Calculate derived metrics
            predicted_daily_output = (predicted_irradiance * self.panel_area *
                                    self.panel_efficiency * self.performance_ratio)
            predicted_annual_output = predicted_daily_output * 365
            predicted_capacity_factor = predicted_daily_output / ((self.panel_rating / 1000) * 24)

            panels_for_1MW = int(1000000 / self.panel_rating)
            predicted_daily_yield_MW = predicted_daily_output * panels_for_1MW
            predicted_annual_yield_MW = predicted_annual_output * panels_for_1MW

            print(f"\nPREDICTION FOR LOCATION ({lat}°, {lon}°):")
            print("-" * 50)
            print(f"  Predicted irradiance: {predicted_irradiance:.2f} kWh/m²/day")
            print(f"  Daily output per panel: {predicted_daily_output:.2f} kWh/day")
            print(f"  Annual output per panel: {predicted_annual_output:.0f} kWh/year")
            print(f"  Capacity factor: {predicted_capacity_factor:.3f} ({predicted_capacity_factor*100:.1f}%)")
            print(f"  Daily yield per MW: {predicted_daily_yield_MW:.0f} kWh/day")
            print(f"  Annual yield per MW: {predicted_annual_yield_MW:.0f} kWh/year")

            return {
                'irradiance': predicted_irradiance,
                'daily_output_per_panel': predicted_daily_output,
                'annual_output_per_panel': predicted_annual_output,
                'capacity_factor': predicted_capacity_factor,
                'daily_yield_per_MW': predicted_daily_yield_MW,
                'annual_yield_per_MW': predicted_annual_yield_MW
            }

        except Exception as e:
            print(f"Error making prediction: {e}")
            return None

    def save_results(self, filename='solar_analysis_results_monthly.csv'):
        """
        Save analysis results to CSV
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No results to save")
            return

        try:
            self.site_summary.to_csv(filename, index=False)
            print(f"Results saved to {filename}")
        except Exception as e:
            print(f"Error saving results: {e}")

    def export_top_sites_kml(self, top_n=10, filename='top_solar_sites.kml'):
        """
        Export top N sites to KML format for Google Earth visualization
        """
        if self.site_summary is None or self.site_summary.empty:
            print("No data to export")
            return

        try:
            top_sites = self.site_summary.head(top_n)

            kml_content = '''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>Top Solar Sites</name>
    <description>Best solar energy potential locations</description>
'''

            for _, site in top_sites.iterrows():
                kml_content += f'''    <Placemark>
      <name>Rank {site['rank']} - Solar Site</name>
      <description>
        <![CDATA[
        <b>Location:</b> ({site['LAT']:.1f}°, {site['LON']:.1f}°)<br/>
        <b>Irradiance:</b> {site['avg_irradiance_kwh']:.2f} kWh/m²/day<br/>
        <b>Capacity Factor:</b> {site['capacity_factor']*100:.1f}%<br/>
        <b>Daily Output per Panel:</b> {site['daily_output_per_panel']:.2f} kWh/day<br/>
        <b>Annual Yield per MW:</b> {site['annual_yield_per_MW_nameplate']:.0f} kWh/year<br/>
        <b>Solar Score:</b> {site['solar_score']:.1f}/100
        ]]>
      </description>
      <Point>
        <coordinates>{site['LON']},{site['LAT']},0</coordinates>
      </Point>
    </Placemark>
'''

            kml_content += '''  </Document>
</kml>'''

            with open(filename, 'w') as f:
                f.write(kml_content)

            print(f"Top {top_n} sites exported to {filename} for Google Earth visualization")

        except Exception as e:
            print(f"Error exporting KML: {e}")

# Main execution function
def main():
    try:
        print("Starting Solar Plant Analysis...")
        print("Expected CSV format: NASA POWER Monthly Data")

        # Use your uploaded file
        csv_file_path = 'POWER_Regional_Monthly_2023_2024.csv'
        analyzer = SolarPlantAnalyzer(csv_file_path)

        # Check if data was loaded successfully
        if analyzer.data is None or analyzer.data.empty:
            print("No data loaded. Please check the file path and format.")
            return

        print(f"Successfully loaded {len(analyzer.data)} data points")
        print(f"Using 320W panels with 23% efficiency and 85% performance ratio")

        # Perform analysis
        print("\n" + "="*60)
        print("PERFORMING SOLAR ANALYSIS...")
        print("="*60)

        analyzer.calculate_site_metrics()
        analyzer.calculate_solar_potential()
        analyzer.rank_sites()

        # Train prediction model
        if analyzer.site_summary is not None and len(analyzer.site_summary) > 3:
            print("\nTraining machine learning model...")
            r2, rmse = analyzer.train_prediction_model('random_forest')

        # Generate visualizations
        print("\nGenerating visualizations...")
        analyzer.visualize_results()

        # Generate comprehensive report
        analyzer.generate_report()

        # Save results
        analyzer.save_results('solar_analysis_results_monthly_2023_2024.csv')

        # Export top sites for Google Earth
        analyzer.export_top_sites_kml(10, 'top_10_solar_sites.kml')

        # Example predictions for new locations
        if analyzer.model is not None:
            print("\n" + "="*60)
            print("EXAMPLE PREDICTIONS FOR NEW LOCATIONS:")
            print("="*60)

            # Example Indian locations
            test_locations = [
                (28.6, 77.2),   # Delhi region
                (19.1, 72.9),   # Mumbai region
                (13.1, 80.3),   # Chennai region
                (22.6, 88.4),   # Kolkata region
                (23.0, 72.6),   # Ahmedabad region
            ]

            location_names = ['Delhi', 'Mumbai', 'Chennai', 'Kolkata', 'Ahmedabad']

            for (lat, lon), name in zip(test_locations, location_names):
                print(f"\n{name} Region:")
                result = analyzer.predict_new_location(lat, lon)

        print("\n" + "="*60)
        print("ANALYSIS COMPLETED SUCCESSFULLY!")
        print("="*60)
        print("Files generated:")
        print("  - solar_analysis_results_monthly_2023_2024.csv (detailed results)")
        print("  - top_10_solar_sites.kml (Google Earth visualization)")
        print("\nRecommendation: Use the top-ranked sites for solar plant development.")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

        # Debug information
        print("\nDebugging information:")
        csv_file_path = 'POWER_Regional_Monthly_2023_2024.csv'
        try:
            with open(csv_file_path, 'r') as f:
                lines = f.readlines()[:10]
                print(f"First 10 lines of {csv_file_path}:")
                for i, line in enumerate(lines):
                    print(f"Line {i+1}: {line.strip()}")
        except Exception as file_error:
            print(f"Could not read file: {file_error}")

# Additional utility functions
def quick_analysis(csv_file_path):
    """
    Quick analysis function for immediate results
    """
    analyzer = SolarPlantAnalyzer(csv_file_path)
    if analyzer.data is not None:
        analyzer.calculate_site_metrics()
        analyzer.calculate_solar_potential()
        analyzer.rank_sites()

        print("\nQUICK ANALYSIS RESULTS:")
        print("-" * 40)
        top_3 = analyzer.site_summary.head(3)
        for i, (_, site) in enumerate(top_3.iterrows(), 1):
            print(f"{i}. Location ({site['LAT']:.1f}°, {site['LON']:.1f}°)")
            print(f"   Irradiance: {site['avg_irradiance_kwh']:.2f} kWh/m²/day")
            print(f"   Capacity Factor: {site['capacity_factor']*100:.1f}%")
            print(f"   Daily Yield/MW: {site['daily_yield_per_MW_nameplate']:.0f} kWh")

        return analyzer
    return None

def compare_locations(analyzer, locations):
    """
    Compare multiple locations using the trained model
    """
    if analyzer.model is None:
        print("Model not trained. Please run full analysis first.")
        return

    print("\nLOCATION COMPARISON:")
    print("-" * 60)

    results = []
    for i, (lat, lon, name) in enumerate(locations):
        prediction = analyzer.predict_new_location(lat, lon)
        if prediction:
            results.append((name, lat, lon, prediction))

    # Sort by capacity factor
    results.sort(key=lambda x: x[3]['capacity_factor'], reverse=True)

    print("\nRANKED COMPARISON (by Capacity Factor):")
    for i, (name, lat, lon, pred) in enumerate(results, 1):
        print(f"{i}. {name}: {pred['capacity_factor']*100:.1f}% CF, "
              f"{pred['annual_yield_per_MW']:.0f} kWh/MW/year")

if __name__ == "__main__":
    main()

Starting Solar Plant Analysis...
Expected CSV format: NASA POWER Monthly Data
Loading NASA POWER monthly dataset...
Looking for file: POWER_Regional_Monthly_2023_2024.csv
Error: File 'POWER_Regional_Monthly_2023_2024.csv' not found!
No data loaded. Please check the file path and format.
